In [38]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="os.fork() was called. os.fork() is incompatible with multithreaded code", category=RuntimeWarning)

In [2]:
!pip install --upgrade --quiet transformers peft accelerate bitsandbytes
!pip install evaluate jiwer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.0 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5

In [5]:
!pip install protobuf==4.25.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatibl

In [39]:

import time
import math
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch
import evaluate
from tqdm import tqdm


In [62]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from peft import LoraConfig, get_peft_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Load processor first
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")

# 2. Load the base model
base_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten")

# --- CRITICAL FIX 1: Resize the Decoder Head (Solves RuntimeError) ---
base_model.decoder.resize_token_embeddings(len(processor.tokenizer))
# --- END CRITICAL FIX 1 ---


# 3. Set required generation config IDs
base_model.config.decoder_start_token_id = processor.tokenizer.bos_token_id
base_model.config.pad_token_id = processor.tokenizer.pad_token_id
base_model.config.eos_token_id = processor.tokenizer.eos_token_id
base_model.config.vocab_size = base_model.decoder.config.vocab_size


# --- CRITICAL FIX 2: Patch Base Decoder Config (Solves Checkpoint Warning) ---
# Ensure the configuration object for the DECODER itself has the vocab_size set.
base_model.decoder.config.vocab_size = len(processor.tokenizer)
print("Patched base decoder config.")
# --- END CRITICAL FIX 2 ---


# 4. Define and apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Apply LoRA to base_model to create the PEFT-wrapped model
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 5. Final safety patches (on PEFT model config)
model.config.vocab_size = processor.tokenizer.vocab_size
model.config.decoder.vocab_size = processor.tokenizer.vocab_size 
print(f"Patched PEFT model config with vocab_size: {model.config.vocab_size}")

# 6. Move the PEFT-wrapped model to the device
model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Patched base decoder config.
trainable params: 104,448 || all params: 61,679,616 || trainable%: 0.1693
Patched PEFT model config with vocab_size: 64002


PeftModel(
  (base_model): LoraModel(
    (model): VisionEncoderDecoderModel(
      (encoder): DeiTModel(
        (embeddings): DeiTEmbeddings(
          (patch_embeddings): DeiTPatchEmbeddings(
            (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): DeiTEncoder(
          (layer): ModuleList(
            (0-11): 12 x DeiTLayer(
              (attention): DeiTAttention(
                (attention): DeiTSelfAttention(
                  (query): Linear(in_features=384, out_features=384, bias=True)
                  (key): Linear(in_features=384, out_features=384, bias=True)
                  (value): Linear(in_features=384, out_features=384, bias=True)
                )
                (output): DeiTSelfOutput(
                  (dense): Linear(in_features=384, out_features=384, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
        

In [63]:
# If model is PEFT-wrapped, ensuring it reports trainable params:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / Total params: {total:,}")

# training hyperparams (tweak as needed)
num_epochs = 2
accum_steps = 8
learning_rate = 5e-5
grad_clip = 1.0
save_every_n_steps = 500
output_dir = "./trocr_lora_manual"

# optimizer
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# mixed precision scaler (Fix for FutureWarning, and optimization)
use_amp = torch.cuda.is_available()
# Use torch.amp.GradScaler instead of the deprecated torch.cuda.amp.GradScaler
scaler = torch.amp.GradScaler(enabled=use_amp) if use_amp else None

# metrics
wer_metric = evaluate.load("wer")

Trainable params: 104,448 / Total params: 61,679,616


In [64]:

# helper: generate predictions for a batch of pixel_values (tensor on device)
def generate_preds(batch_pixel_values, max_length=64, num_beams=4):
    # batch_pixel_values: tensor (B, C, H, W)
    model.eval()
    with torch.no_grad():
        # model.generate expects pixel_values on the device
        generated_ids = model.generate(
            pixel_values=batch_pixel_values,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            decoder_start_token_id=processor.tokenizer.bos_token_id,
        )
        preds = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return preds

# eval function (computes wer + exact match)
def evaluate_epoch(eval_loader, max_length=64, num_beams=4, limit=None):
    model.eval()
    pred_texts = []
    true_texts = []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(eval_loader, desc="Eval")):
            # batch keys: pixel_values, labels (as in collate_fn)
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            # decode true labels to text (convert -100 -> pad token id first)
            labels_for_decoding = labels.clone()
            labels_for_decoding[labels_for_decoding == -100] = processor.tokenizer.pad_token_id
            label_strs = processor.batch_decode(labels_for_decoding, skip_special_tokens=True)
            true_texts.extend([s.strip() for s in label_strs])

            preds = generate_preds(pixel_values, max_length=max_length, num_beams=num_beams)
            pred_texts.extend([s.strip() for s in preds])

            if limit and i >= limit:
                break

    # compute metrics
    wer = wer_metric.compute(predictions=pred_texts, references=true_texts)
    exact = sum(1 for a,b in zip(pred_texts, true_texts) if a.strip()==b.strip()) / max(1, len(pred_texts))
    return {"wer": wer, "exact_match": exact, "preds": pred_texts[:5], "refs": true_texts[:5]}


In [50]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import TrOCRProcessor

# --- REVISED HandwritingDataset CLASS ---
class HandwritingDataset(Dataset):
    def __init__(self, df, processor, max_target_length=64):
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row["full_path"]
        label = row["IDENTITY"]

        image = Image.open(img_path).convert("RGB")

        # 🚀 FIX 1: Separate image processing (feature_extractor) and text tokenization (tokenizer)
        # This prevents the 'Unused or unrecognized kwargs' warnings.
        pixel_values = self.processor.feature_extractor(
            image,
            return_tensors="pt"
        ).pixel_values.squeeze(0)

        # Tokenize text/labels separately
        labels = self.processor.tokenizer(
            label,
            padding="max_length",
            max_length=self.max_target_length,
            truncation=True,
            return_tensors="pt"
        ).input_ids.squeeze(0)

        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": labels
        }

In [53]:
import os
import re
import pandas as pd

def load_and_clean_csv(csv_path, img_dir):
    # Load data
    df = pd.read_csv(csv_path)

    # 1. Path construction (Fix for File Not Found / cross-platform compatibility)
    # Use os.path.join for robust path building
    df["full_path"] = df["FILENAME"].apply(lambda x: os.path.join(img_dir, x))

    # Ensure IDENTITY is string and filter empty labels
    df["IDENTITY"] = df["IDENTITY"].astype(str)
    df = df[df["IDENTITY"].str.strip().str.len() > 0].copy()

    # Normalize IDENTITY: lowercase, remove non-alphanumeric (as per original intent)
    # Using 'copy()' to avoid a SettingWithCopyWarning
    df["IDENTITY"] = df["IDENTITY"].str.lower().str.replace(r"[^a-z0-9\s]", "", regex=True).str.strip()

    # 2. Remove rows where the image file does not exist (Fix for "File not found during training")
    # 3. Reset index after filtering to prevent `ValueError: Item wrong length`
    df = df.loc[df["full_path"].apply(os.path.exists)].reset_index(drop=True)

    print(f"Loaded and cleaned {len(df)} lines from {csv_path}")

    return df

# Example usage (assuming your original path variables are set correctly):
# train_df = load_and_clean_csv(train_csv, train_img_dir)
# valid_df = load_and_clean_csv(valid_csv, valid_img_dir)
# test_df = load_and_clean_csv(test_csv, test_img_dir)

# ... followed by dataset/dataloader creation ...


path = "/kaggle/input/handwriting-recognitionocr"

train_csv = f"{path}/CSV/written_name_train.csv"
valid_csv = f"{path}/CSV/written_name_test.csv"
test_csv  = f"{path}/CSV/written_name_validation.csv"

train_img_dir = f"{path}/train_v2/train"
valid_img_dir = f"{path}/test_v2/test"
test_img_dir  = f"{path}/validation_v2/validation"

# Cleaned DataFrames
train_df = load_and_clean_csv(train_csv, train_img_dir)
valid_df = load_and_clean_csv(valid_csv, valid_img_dir)
test_df  = load_and_clean_csv(test_csv,  test_img_dir)


train_dataset = HandwritingDataset(train_df, processor)
valid_dataset = HandwritingDataset(valid_df,  processor)
test_dataset  = HandwritingDataset(test_df,   processor)

def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    return torch.utils.data.default_collate(batch)



train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_skip_none,
)

eval_loader = DataLoader(
    valid_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_skip_none,
)


Loaded and cleaned 330961 lines from /kaggle/input/handwriting-recognitionocr/CSV/written_name_train.csv
Loaded and cleaned 41370 lines from /kaggle/input/handwriting-recognitionocr/CSV/written_name_test.csv
Loaded and cleaned 41370 lines from /kaggle/input/handwriting-recognitionocr/CSV/written_name_validation.csv


In [ ]:
import time
import os
import torch
from tqdm.auto import tqdm

# Ensure all external variables (num_epochs, optimizer, scaler, etc.) are defined 
# in the cells above this one.

global_step = 0
best_wer = float("inf")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    start = time.time()
    
    # CRITICAL FIX: leave=False keeps the progress bar output clean
    pbar = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", leave=False)
    
    optimizer.zero_grad()

    for step, batch in enumerate(train_loader):
        pixel_values = batch["pixel_values"].to(device)      # (B, C, H, W)
        labels = batch["labels"].to(device)                  # (B, L) with -100 for padding

        # Use autocast from torch.cuda.amp directly (standard practice)
        with torch.cuda.amp.autocast(enabled=use_amp):
            # forward pass (VisionEncoderDecoder model accepts pixel_values and labels)
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss / accum_steps  # average for accumulation

        scaler.scale(loss).backward()
        epoch_loss += loss.item() * accum_steps  # multiply back

        if (step + 1) % accum_steps == 0:
            # gradient clipping (unscale first)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1

            if global_step % 50 == 0:
                # Use current global step for loss tracking
                # NOTE: Ensure global_step starts at 0 and increments correctly
                pbar.set_postfix({"loss": f"{epoch_loss/global_step:.4f}"})

        # NO INTERMEDIATE SAVE BLOCK HERE (Removed the problematic global_step % save_every_n_steps == 0)
        
        pbar.update(1) # Update the progress bar for every batch
    
    # --- END OF BATCH LOOP (DEDENTED) ---
    pbar.close()
    
    epoch_time = time.time() - start
    avg_loss = epoch_loss / max(1, len(train_loader))
    print(f"Epoch {epoch+1} done — avg_loss: {avg_loss:.4f} — time: {epoch_time/60:.2f} min")

    # evaluate on full validation set
    metrics = evaluate_epoch(eval_loader, max_length=64, num_beams=4, limit=None)
    print(f"Validation — WER: {metrics['wer']:.4f}  Exact: {metrics['exact_match']:.4f}")
    print("Sample preds:", metrics["preds"])
    print("Sample refs :", metrics["refs"])

    # save best
    if metrics["wer"] < best_wer:
        best_wer = metrics["wer"]
        os.makedirs(output_dir, exist_ok=True)
        print("New best — saving model to", output_dir)
        
        # *** CRITICAL PATCH: Guaranteed Vocab Size for Saving (Final/Best) ***
        # Apply patch just before saving.
        try:
            model.config.vocab_size = len(processor.tokenizer)
            model.config.decoder.vocab_size = len(processor.tokenizer)
        except:
            pass
            
        # Try the standard save first
        try:
            model.save_pretrained(output_dir)
            processor.save_pretrained(output_dir)
            print("Successfully saved best model!")
        except Exception as e:
            # *** ULTIMATE FALLBACK: Manually save LoRA weights if config save fails ***
            print(f"Failed to save model: {e}. Attempting manual LoRA adapter save...")
            try:
                # This saves only the small, essential adapter weights, which is all you need for PEFT.
                torch.save(model.state_dict(), os.path.join(output_dir, "adapter_weights_manual.pt"))
                processor.save_pretrained(output_dir) # Still save the processor config
                print(f"✅ SUCCESS: Manual LoRA adapter weights saved to {output_dir}/adapter_weights_manual.pt")
            except Exception as manual_e:
                print(f"CRITICAL ERROR: Failed manual save. Your weights are only in GPU memory. {manual_e}")

Epoch 1/2:   0%|          | 0/20686 [00:00<?, ?batch/s]

In [ ]:
import os
import torch

# Define the directory where you want to save the checkpoint
# (Use a new directory name just in case)
FINAL_SAVE_DIR = "./trocr_lora_final_save"
os.makedirs(FINAL_SAVE_DIR, exist_ok=True)

print("Attempting manual save...")

# *** Final, Most Aggressive Configuration Patch ***
# Patch the attributes one final time directly on the live model configuration 
# before calling the save function.
try:
    model.config.vocab_size = len(processor.tokenizer)
    model.config.decoder.vocab_size = len(processor.tokenizer)
    print("Configuration patched successfully.")
except Exception as e:
    print(f"Warning: Configuration patch failed: {e}")

# --- CRITICAL SAVE STEP ---
# 1. Save the PEFT adapter weights (the LoRA changes)
model.save_pretrained(FINAL_SAVE_DIR)
# 2. Save the processor (tokenizer and feature extractor)
processor.save_pretrained(FINAL_SAVE_DIR)

print(f"\n✅ SUCCESS! Your model progress has been saved to: {FINAL_SAVE_DIR}")
print("You can load the model back using: model = PeftModel.from_pretrained(base_model, FINAL_SAVE_DIR)")
print("Training finished. Best WER:", best_wer)
